In [1]:
import os
import random
import sys
 
import tensorflow as tf
import cv2
from PIL import Image

In [14]:
def _int64_feature(value):
    """Wrapper for insert int64 feature into Example proto."""
    if not isinstance(value, list):
        value = [value]
    return tf.train.Feature(int64_list=tf.train.Int64List(value=value))
 
def _float_feature(value):
    """Wrapper for insert float features into Example proto."""
    if not isinstance(value, list):
        value = [value]
    return tf.train.Feature(float_list=tf.train.FloatList(value=value))
 
def _bytes_feature(value):
    """Wrapper for insert bytes features into Example proto."""
    if not isinstance(value, list):
        value = [value]
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=value))

In [18]:
def _process_image_withoutcoder(filename):
    """
    利用cv2将filename指向的图片tostring
    """
 
    print("filename",filename)
    image = cv2.imread(filename)
 
    # transform data into string format
    image_data = image.tostring()
    assert len(image.shape) == 3
    height = image.shape[0]
    width = image.shape[1]
    assert image.shape[2] == 3
    # return string data and initial height and width of the image
    return image_data, height, width
 

def _convert_to_example_simple(image_example, image_buffer):
    """
        covert to tfrecord file
    Parameter
    ------------
        image_example: dict, an image example
        image_buffer: string, JPEG encoding of RGB image
    Return
    -----------
        Example proto
    """
    class_label = image_example['label']
 
    bbox = image_example['bbox']
    roi = [bbox['xmin'], bbox['ymin'], bbox['xmax'], bbox['ymax']]
    # landmark = [bbox['xlefteye'],bbox['ylefteye'],bbox['xrighteye'],bbox['yrighteye'],bbox['xnose'],bbox['ynose'],
    #             bbox['xleftmouth'],bbox['yleftmouth'],bbox['xrightmouth'],bbox['yrightmouth']]
 
    example = tf.train.Example(features=tf.train.Features(feature={
        'image/encoded': _bytes_feature(image_buffer),
        'image/label': _int64_feature(class_label),
        'image/roi': _float_feature(roi),
        # 'image/landmark': _float_feature(landmark)
    }))
    return example
 
# 从图片和注释文件里加载数据并将其添加到TFRecord里
# 参数（变量）：filename:存有数据的字典；tfrecord_writer:用来写入TFRecord的writer
 
def _add_to_tfrecord(filename, image_example, tfrecord_writer):
    # print('---', filename)
 
    # imaga_data:转化为字符串的图片
    # height:图片原始高度
    # width:图片原始宽度
    # image_example：包含图片信息的字典
    # print(filename)
    image_data, height, width = _process_image_withoutcoder(filename)
    example = _convert_to_example_simple(image_example, image_data)
    tfrecord_writer.write(example.SerializeToString())  # 将imaga_data转化到image_example中并写入tfrecord
 
 
def _get_output_filename(output_dir,net):
    # 定义一下输出的文件名
 
    # return '%s/%s_%s_%s.tfrecord' % (output_dir, name, net, st)
    # st = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())
    # time.strftime() 函数接收以时间元组，并返回以可读字符串表示的当地时间，格式由参数format决定:time.strftime(format[, t]),用来输出当前时间
    # 返回的是'../../DATA/imglists/PNet/train_PNet_landmark.tfrecord'
    return 'data/%s/train_%s_landmark.tfrecord' % (output_dir,net)


In [21]:
def run(dataset_dir,net,output_dir,shuffling=False):
    """
    运行转换操作
    Args:
      dataset_dir: 数据集所在的数据集目录
      output_dir: 输出目录
    """
 
    # tfrecord name
    tf_filename = _get_output_filename(output_dir,net)  # '../../DATA/imglists/PNet/train_PNet_landmark.tfrecord'
 
    if tf.io.gfile.exists(tf_filename):  # tf.io.gfile模块提供了文件操作的API,包括文件的读取、写入、删除、复制等等
        print('Dataset files already exist. Exiting without re-creating them.')  # 判断是否存在同名文件
        return
 
    # 获得数据集，并打乱顺序
    dataset = get_dataset(dataset_dir)
    #print(dataset)
    # filenames = dataset['filename']
    if shuffling:
        tf_filename = tf_filename + '_shuffle'
        # random.seed(12345454)
        random.shuffle(dataset)  # 打乱dataset数据集的顺序
 
    # Process dataset files.
    # write the data to tfrecord
    print('lala')
    with tf.io.TFRecordWriter(tf_filename) as tfrecord_writer:
        for i, image_example in enumerate(dataset):  # 读取dataset的索引和内容
            if (i + 1) % 1 == 0:
                sys.stdout.write('\r>> %d/%d images has been converted' % (
                i + 1, len(dataset)))  # 输出“x00/ len(dataset) images has been converted”
            sys.stdout.flush()  # 以一定间隔时间刷新输出
            filename = image_example['filename']  # 赋值
            _add_to_tfrecord(filename, image_example, tfrecord_writer)
    # 最后，编写标签文件
    # labels_to_class_names = dict(zip(range(len(_CLASS_NAMES)), _CLASS_NAMES))
    # dataset_utils.write_label_file(labels_to_class_names, dataset_dir)
    print('\nFinished converting the MTCNN dataset!')
 
 
def get_dataset(dir):
    # 获取文件名字，标签和注释
    item = 'label-train%s.txt'%(dir)
 
    dataset_dir = os.path.join('data',dir, item)  # dataset_dir = '../../DATA/imglists/PNet/train_PNet_landmark.txt'
    # print(dataset_dir)
    imagelist = open(dataset_dir, 'r')  # 以只读的形式打开train_PNet_landmark.txt，并传入imagelist里面
 
    dataset = []  # 新建列表
    for line in imagelist.readlines():  # 按行读取imagelist里面的内容
        info = line.strip().split(' ')  # .strip().split()去除每一行首尾空格并且以空格为分隔符读取内容到info里面
        data_example = dict()  # 新建字典
        bbox = dict()
        data_example['filename'] = 'data/' + info[0]  # filename=info[0]
        # print(data_example['filename'])
        data_example['label'] = int(info[1])  # label=info[1]，info[1]的值有四种可能，1，0，-1，-2；分别对应着正、负、无关、关键点样本
        bbox['xmin'] = 0  # 初始化bounding box的值
        bbox['ymin'] = 0
        bbox['xmax'] = 0
        bbox['ymax'] = 0
        # bbox['xlefteye'] = 0  # 初始化人脸坐标的值
        # bbox['ylefteye'] = 0
        # bbox['xrighteye'] = 0
        # bbox['yrighteye'] = 0
        # bbox['xnose'] = 0
        # bbox['ynose'] = 0
        # bbox['xleftmouth'] = 0
        # bbox['yleftmouth'] = 0
        # bbox['xrightmouth'] = 0
        # bbox['yrightmouth'] = 0
        if len(info) == 6:  # 当info的长度等于6时，表示此时的info是正样本或者无关样本
            bbox['xmin'] = float(info[2])
            bbox['ymin'] = float(info[3])
            bbox['xmax'] = float(info[4])
            bbox['ymax'] = float(info[5])
        # if len(info) == 12:  # 当info的长度等于12时，表示此时的info是landmark样本
        #     bbox['xlefteye'] = float(info[2])
        #     bbox['ylefteye'] = float(info[3])
        #     bbox['xrighteye'] = float(info[4])
        #     bbox['yrighteye'] = float(info[5])
        #     bbox['xnose'] = float(info[6])
        #     bbox['ynose'] = float(info[7])
        #     bbox['xleftmouth'] = float(info[8])
        #     bbox['yleftmouth'] = float(info[9])
        #     bbox['xrightmouth'] = float(info[10])
        #     bbox['yrightmouth'] = float(info[11])
 
        data_example['bbox'] = bbox  # 将bounding box值传入字典
        dataset.append(data_example)  # 将data_example字典内容传入列表dataset
 
    return dataset  # 返回的是dataset，datase是个列表，但里面每个元素都是一个字典，每个字典都含有3个key，分别是filename、label和bounding box

In [22]:

if __name__ == '__main__':
    dir = '24'
    net = 'RNet'
    output_directory = '24'
    run(dir,net,output_directory,shuffling=True)

lala
>> 1/7977 images has been convertedfilename data/24/negative/1060.jpg
>> 2/7977 images has been convertedfilename data/24/positive/8.jpg
>> 3/7977 images has been convertedfilename data/24/positive/893.jpg
>> 4/7977 images has been convertedfilename data/24/positive/676.jpg
>> 5/7977 images has been convertedfilename data/24/positive/627.jpg
>> 6/7977 images has been convertedfilename data/24/part/225.jpg
>> 7/7977 images has been convertedfilename data/24/negative/2881.jpg
>> 8/7977 images has been convertedfilename data/24/negative/321.jpg
>> 9/7977 images has been convertedfilename data/24/negative/2758.jpg
>> 10/7977 images has been convertedfilename data/24/negative/3664.jpg
>> 11/7977 images has been convertedfilename data/24/negative/3627.jpg
>> 12/7977 images has been convertedfilename data/24/negative/2571.jpg
>> 13/7977 images has been convertedfilename data/24/part/812.jpg
>> 14/7977 images has been convertedfilename data/24/negative/3025.jpg
>> 15/7977 images has been 

/opt/anaconda3/envs/envTF22/lib/python3.7/site-packages/ipykernel_launcher.py:10: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  # Remove the CWD from sys.path while we load stuff.


filename data/24/positive/906.jpg
>> 108/7977 images has been convertedfilename data/24/negative/1581.jpg
>> 109/7977 images has been convertedfilename data/24/negative/5152.jpg
>> 110/7977 images has been convertedfilename data/24/negative/705.jpg
>> 111/7977 images has been convertedfilename data/24/negative/1889.jpg
>> 112/7977 images has been convertedfilename data/24/positive/553.jpg
>> 113/7977 images has been convertedfilename data/24/negative/1218.jpg
>> 114/7977 images has been convertedfilename data/24/negative/1984.jpg
>> 115/7977 images has been convertedfilename data/24/negative/4686.jpg
>> 116/7977 images has been convertedfilename data/24/positive/691.jpg
>> 117/7977 images has been convertedfilename data/24/negative/5583.jpg
>> 118/7977 images has been convertedfilename data/24/negative/261.jpg
>> 119/7977 images has been convertedfilename data/24/part/709.jpg
>> 120/7977 images has been convertedfilename data/24/negative/2466.jpg
>> 121/7977 images has been convertedfi

>> 223/7977 images has been convertedfilename data/24/negative/1730.jpg
>> 224/7977 images has been convertedfilename data/24/negative/1025.jpg
>> 225/7977 images has been convertedfilename data/24/negative/5740.jpg
>> 226/7977 images has been convertedfilename data/24/part/256.jpg
>> 227/7977 images has been convertedfilename data/24/negative/5645.jpg
>> 228/7977 images has been convertedfilename data/24/negative/1739.jpg
>> 229/7977 images has been convertedfilename data/24/positive/498.jpg
>> 230/7977 images has been convertedfilename data/24/negative/5745.jpg
>> 231/7977 images has been convertedfilename data/24/negative/360.jpg
>> 232/7977 images has been convertedfilename data/24/part/557.jpg
>> 233/7977 images has been convertedfilename data/24/negative/4336.jpg
>> 234/7977 images has been convertedfilename data/24/negative/3926.jpg
>> 235/7977 images has been convertedfilename data/24/negative/5410.jpg
>> 236/7977 images has been convertedfilename data/24/negative/167.jpg
>> 23

>> 338/7977 images has been convertedfilename data/24/negative/5699.jpg
>> 339/7977 images has been convertedfilename data/24/negative/5946.jpg
>> 340/7977 images has been convertedfilename data/24/negative/3690.jpg
>> 341/7977 images has been convertedfilename data/24/positive/311.jpg
>> 342/7977 images has been convertedfilename data/24/negative/1890.jpg
>> 343/7977 images has been convertedfilename data/24/part/906.jpg
>> 344/7977 images has been convertedfilename data/24/negative/2235.jpg
>> 345/7977 images has been convertedfilename data/24/negative/5724.jpg
>> 346/7977 images has been convertedfilename data/24/negative/3813.jpg
>> 347/7977 images has been convertedfilename data/24/negative/4272.jpg
>> 348/7977 images has been convertedfilename data/24/positive/538.jpg
>> 349/7977 images has been convertedfilename data/24/positive/655.jpg
>> 350/7977 images has been convertedfilename data/24/negative/2477.jpg
>> 351/7977 images has been convertedfilename data/24/negative/3730.jpg


...

>> 7281/7977 images has been convertedfilename data/24/negative/4635.jpg
>> 7282/7977 images has been convertedfilename data/24/part/8.jpg
>> 7283/7977 images has been convertedfilename data/24/negative/4553.jpg
>> 7284/7977 images has been convertedfilename data/24/negative/450.jpg
>> 7285/7977 images has been convertedfilename data/24/positive/799.jpg
>> 7286/7977 images has been convertedfilename data/24/positive/164.jpg
>> 7287/7977 images has been convertedfilename data/24/positive/176.jpg
>> 7288/7977 images has been convertedfilename data/24/negative/3240.jpg
>> 7289/7977 images has been convertedfilename data/24/negative/1530.jpg
>> 7290/7977 images has been convertedfilename data/24/negative/615.jpg
>> 7291/7977 images has been convertedfilename data/24/positive/235.jpg
>> 7292/7977 images has been convertedfilename data/24/negative/1614.jpg
>> 7293/7977 images has been convertedfilename data/24/negative/3473.jpg
>> 7294/7977 images has been convertedfilename data/24/negative/

>> 7395/7977 images has been convertedfilename data/24/part/869.jpg
>> 7396/7977 images has been convertedfilename data/24/negative/5491.jpg
>> 7397/7977 images has been convertedfilename data/24/negative/3750.jpg
>> 7398/7977 images has been convertedfilename data/24/negative/541.jpg
>> 7399/7977 images has been convertedfilename data/24/part/732.jpg
>> 7400/7977 images has been convertedfilename data/24/part/915.jpg
>> 7401/7977 images has been convertedfilename data/24/positive/823.jpg
>> 7402/7977 images has been convertedfilename data/24/part/599.jpg
>> 7403/7977 images has been convertedfilename data/24/negative/1483.jpg
>> 7404/7977 images has been convertedfilename data/24/negative/5899.jpg
>> 7405/7977 images has been convertedfilename data/24/negative/3020.jpg
>> 7406/7977 images has been convertedfilename data/24/positive/185.jpg
>> 7407/7977 images has been convertedfilename data/24/negative/2545.jpg
>> 7408/7977 images has been convertedfilename data/24/negative/4160.jpg
>

>> 7509/7977 images has been convertedfilename data/24/negative/1289.jpg
>> 7510/7977 images has been convertedfilename data/24/negative/4468.jpg
>> 7511/7977 images has been convertedfilename data/24/positive/935.jpg
>> 7512/7977 images has been convertedfilename data/24/negative/353.jpg
>> 7513/7977 images has been convertedfilename data/24/negative/1034.jpg
>> 7514/7977 images has been convertedfilename data/24/negative/5297.jpg
>> 7515/7977 images has been convertedfilename data/24/negative/776.jpg
>> 7516/7977 images has been convertedfilename data/24/negative/933.jpg
>> 7517/7977 images has been convertedfilename data/24/negative/5804.jpg
>> 7518/7977 images has been convertedfilename data/24/negative/819.jpg
>> 7519/7977 images has been convertedfilename data/24/positive/611.jpg
>> 7520/7977 images has been convertedfilename data/24/negative/3412.jpg
>> 7521/7977 images has been convertedfilename data/24/negative/4552.jpg
>> 7522/7977 images has been convertedfilename data/24/ne

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



filename data/24/positive/870.jpg
>> 7931/7977 images has been convertedfilename data/24/negative/1586.jpg
>> 7932/7977 images has been convertedfilename data/24/negative/2879.jpg
>> 7933/7977 images has been convertedfilename data/24/negative/248.jpg
>> 7934/7977 images has been convertedfilename data/24/negative/4155.jpg
>> 7935/7977 images has been convertedfilename data/24/negative/2873.jpg
>> 7936/7977 images has been convertedfilename data/24/negative/1829.jpg
>> 7937/7977 images has been convertedfilename data/24/negative/1591.jpg
>> 7938/7977 images has been convertedfilename data/24/positive/551.jpg
>> 7939/7977 images has been convertedfilename data/24/negative/1611.jpg
>> 7940/7977 images has been convertedfilename data/24/negative/2613.jpg
>> 7941/7977 images has been convertedfilename data/24/negative/25.jpg
>> 7942/7977 images has been convertedfilename data/24/part/823.jpg
>> 7943/7977 images has been convertedfilename data/24/positive/660.jpg
>> 7944/7977 images has bee